In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
""" Opening and viewing an .obj file """
obj_data = "human_nobg.obj"
mesh = o3d.io.read_triangle_mesh(obj_data)

In [3]:
""" Visualizing point cloud """
mesh.compute_vertex_normals()
vertices = np.asarray(mesh.vertices)
o3d.visualization.draw_geometries([mesh])
# pcd = mesh.sample_points_poisson_disk(len(vertices)//2)
o3d.visualization.draw_geometries([pcd])

In [11]:
""" voxel sampling """
downpcd = pcd.voxel_down_sample(voxel_size = 0.03)
o3d.visualization.draw_geometries([downpcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [15]:
""" vertex normal estimation """
downpcd.estimate_normals(
    search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
o3d.visualization.draw_geometries([downpcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [10]:
vert = vertices
min_vert, max_vert = vert.min(axis=0), vert.max(axis=0)
cube = o3d.geometry.TriangleMesh.create_box()
cube.scale(0.001, center=cube.get_center())
cube.translate(
    (
        np.random.uniform(min_vert[0], max_vert[0]),
        np.random.uniform(min_vert[1], max_vert[1]),
        np.random.uniform(min_vert[2], max_vert[2]),
    ),
    relative=False,
)
cube.compute_vertex_normals()
print("Show input mesh")
o3d.visualization.draw_geometries([cube])

Show input mesh


In [11]:
""" connected components """
print("Generate data")
# mesh = o3dtut.get_bunny_mesh().subdivide_midpoint(number_of_iterations=2)
vert = vertices
min_vert, max_vert = vert.min(axis=0), vert.max(axis=0)
for _ in range(30):
    cube = o3d.geometry.TriangleMesh.create_box()
    cube.scale(0.005, center=cube.get_center())
    cube.translate(
        (
            np.random.uniform(min_vert[0], max_vert[0]),
            np.random.uniform(min_vert[1], max_vert[1]),
            np.random.uniform(min_vert[2], max_vert[2]),
        ),
        relative=False,
    )
    mesh += cube
mesh.compute_vertex_normals()
print("Show input mesh")
o3d.visualization.draw_geometries([mesh])

Generate data
Show input mesh


In [12]:
print("Cluster connected triangles")
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
            triangle_clusters, cluster_n_triangles, cluster_area = (
                mesh.cluster_connected_triangles())
triangle_clusters = np.asarray(triangle_clusters)
cluster_n_triangles = np.asarray(cluster_n_triangles)
cluster_area = np.asarray(cluster_area)

Cluster connected triangles
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=41


In [13]:
print("Show mesh with small clusters removed")
mesh_0 = copy.deepcopy(mesh)
triangles_to_remove = cluster_n_triangles[triangle_clusters] < 100
mesh_0.remove_triangles_by_mask(triangles_to_remove)
o3d.visualization.draw_geometries([mesh_0])

Show mesh with small clusters removed
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [14]:
print("Show largest cluster")
mesh_1 = copy.deepcopy(mesh)
largest_cluster_idx = cluster_n_triangles.argmax()
triangles_to_remove = triangle_clusters != largest_cluster_idx
mesh_1.remove_triangles_by_mask(triangles_to_remove)
o3d.visualization.draw_geometries([mesh_1])

Show largest cluster


In [17]:
o3d.io.write_triangle_mesh("copy_of_crate.obj",
                               mesh_1,
                               write_triangle_uvs=True)

[Open3D WARNING] Write OBJ can not include triangle normals.


True

In [19]:
""" Bounding volumes """
aabb = pcd.get_axis_aligned_bounding_box()
aabb.color = (1,0,0)
obb = pcd.get_oriented_bounding_box()
obb.color = (0,1,0)
o3d.visualization.draw_geometries([pcd, aabb, obb])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


POINT CLOUD DEMO

In [2]:
print("Load a polygon volume and use it to crop the original point cloud")
demo_crop_data = o3d.data.DemoCropPointCloud()
pcd = o3d.io.read_point_cloud(demo_crop_data.point_cloud_path)
vol = o3d.visualization.read_selection_polygon_volume(demo_crop_data.cropped_json_path)
chair = vol.crop_point_cloud(pcd)
o3d.visualization.draw_geometries([chair],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

Load a polygon volume and use it to crop the original point cloud
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
